In [ ]:
!nvidia-smi

Mon Sep 19 15:55:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os 
import pandas as pd 
import numpy as np 
import sklearn 
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, Input, Flatten, Dense, Dropout
import pickle


In [ ]:
train_list = os.listdir("/content/drive/MyDrive/Interface_E2/CNN_CV_dataset/train")
train_list

['dog.5467.jpg',
 'dog.6191.jpg',
 'dog.8157.jpg',
 'dog.1629.jpg',
 'dog.3980.jpg',
 'dog.1601.jpg',
 'dog.155.jpg',
 'dog.6146.jpg',
 'dog.6620.jpg',
 'dog.7516.jpg',
 'dog.4023.jpg',
 'dog.169.jpg',
 'dog.9249.jpg',
 'dog.12203.jpg',
 'dog.5315.jpg',
 'dog.11044.jpg',
 'dog.3758.jpg',
 'dog.2334.jpg',
 'dog.5329.jpg',
 'dog.11722.jpg',
 'dog.4989.jpg',
 'dog.11911.jpg',
 'dog.4745.jpg',
 'dog.1832.jpg',
 'dog.141.jpg',
 'dog.2446.jpg',
 'dog.3016.jpg',
 'dog.6608.jpg',
 'dog.8143.jpg',
 'dog.2485.jpg',
 'dog.10414.jpg',
 'dog.7264.jpg',
 'dog.800.jpg',
 'dog.5301.jpg',
 'dog.3764.jpg',
 'dog.12029.jpg',
 'dog.2136.jpg',
 'dog.4209.jpg',
 'dog.6344.jpg',
 'dog.431.jpg',
 'dog.4751.jpg',
 'dog.11291.jpg',
 'dog.8341.jpg',
 'dog.2122.jpg',
 'dog.7099.jpg',
 'dog.3572.jpg',
 'dog.9088.jpg',
 'dog.6393.jpg',
 'dog.5671.jpg',
 'dog.7066.jpg',
 'dog.1365.jpg',
 'dog.2678.jpg',
 'dog.12001.jpg',
 'dog.10158.jpg',
 'dog.11252.jpg',
 'dog.7700.jpg',
 'dog.5856.jpg',
 'dog.394.jpg',
 'dog.5103

In [ ]:
train_dataframe = pd.DataFrame()
train_dataframe['Filename']=train_list

In [ ]:
train_dataframe

,Filename
0,dog.5467.jpg
1,dog.6191.jpg
2,dog.8157.jpg
3,dog.1629.jpg
4,dog.3980.jpg
...,...
22484,cat.11104.jpg
22485,cat.9499.jpg
22486,cat.1614.jpg
22487,cat.5314.jpg


In [ ]:
train_dataframe['Label'] = np.where(train_dataframe.Filename.str.split(".").str.get(0)=="cat", "cat", "dog")

In [ ]:
train_dataframe

,Filename,Label
0,dog.5467.jpg,dog
1,dog.6191.jpg,dog
2,dog.8157.jpg,dog
3,dog.1629.jpg,dog
4,dog.3980.jpg,dog
...,...,...
22484,cat.11104.jpg,cat
22485,cat.9499.jpg,cat
22486,cat.1614.jpg,cat
22487,cat.5314.jpg,cat


In [ ]:
train_dataframe.to_csv("CNN_CV_dataframe.csv")

In [ ]:
train_data = pd.read_csv('CNN_CV_dataframe.csv')
Y_train = train_data[['Label']]
skf = StratifiedKFold(n_splits=5, random_state=1, shuffle = True)

In [ ]:
preproc_train = ImageDataGenerator(rescale=1.0/255.0, 
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=True)

In [ ]:
def get_model_name(k): 
  return 'model_'+str(k)+'.h5'

In [ ]:
# Une fois les données préparer on peut définir le modèle basique


def basic_cnnx3(): 
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    
    # On doit compiler le modèle en spécifiant le solver = algorithme utilisé pour minimiser l'erreur de prédiction
    # Les valeurs de learning rate et du momentum sont choisies en accord avec les hyperparamètre de VGG16 
    solver = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=solver, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
validation_accuracy = []
validation_loss = []

save_dir = '/content/drive/MyDrive/Interface_E2/save_modelsCV2/'
fold_var = 1 

for train_index, val_index in skf.split(np.zeros(22489), Y_train):
  training_data = train_data.iloc[train_index]
  validation_data = train_data.iloc[val_index]

  train_data_generator = preproc_train.flow_from_dataframe(training_data, 
                                                           directory='/content/drive/MyDrive/Interface_E2/CNN_CV_dataset/train', 
                                                           x_col = "Filename", 
                                                           y_col = "Label", 
                                                           class_mode = 'binary',
                                                           shuffle = True, 
                                                           target_size=(224, 224))
  
  valid_data_generator = preproc_train.flow_from_dataframe(validation_data, 
                                                           directory='/content/drive/MyDrive/Interface_E2/CNN_CV_dataset/train', 
                                                           x_col = "Filename", 
                                                           y_col = 'Label', 
                                                           class_mode = 'binary', 
                                                           shuffle = True, 
                                                           target_size=(224, 224))
  
  model = basic_cnnx3()

  # Creation de callbacks 
  checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
                                                  monitor='val_accuracy', verbose=1, 
                                                  save_best_only=True, mode='max')
  callbacks_list = [checkpoint]


  history = model.fit(train_data_generator, 
                      epochs=30, 
                      callbacks=callbacks_list, 
                      validation_data = valid_data_generator)
  
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  
  # On regarde l'évolution en fonction des epochs 
  epochs = range(1, len(acc)+1)
  
  # On plot l'accuracy de l'entrainement et de la validation par epoch
  plt.plot(epochs, acc, 'bo', label='Train_Acc')
  plt.plot(epochs, val_acc, 'b', label="Val_Acc")
  plt.title('Accuracy sur Train et Validation')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend()
  
  # On enregistre les graphiques
  filename1 = "Train_Val_Acc_BasicCNNx3DACV_fold"+str(fold_var)
  plt.savefig(save_dir + filename1 + '_plot.png')
  plt.close()
  
  
  plt.figure()
  plt.plot(epochs, loss, 'bo', label='Train_Loss')
  plt.plot(epochs, val_loss, 'b', label="Val_Loss" )
  plt.title('Loss sur Train et Validation')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend()
  filename2 = "Train_Val_Loss_BasicCNNx3DACV_fold"+str(fold_var)
  plt.savefig(save_dir + filename2 + '_plot.png')
  plt.close()
  

  # On charge le meilleur modèle pour en évaluer les performances
  model.load_weights("/content/drive/MyDrive/Interface_E2/save_modelsCV2/model_"+str(fold_var)+".h5")

  results = model.evaluate(valid_data_generator)
  results = dict(zip(model.metrics_names, results))

  validation_accuracy.append(results['accuracy'])
  validation_loss.append(results['loss'])

  tf.keras.backend.clear_session()

  fold_var+=1

Found 17991 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1 invalid image filename(s) in x_col="Filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 4497 validated image filenames belonging to 2 classes.
Epoch 1/30
563/563 [==============================] - ETA: 0s - loss: 0.6887 - accuracy: 0.5951
Epoch 1: val_accuracy improved from -inf to 0.65666, saving model to /content/drive/MyDrive/Interface_E2/save_modelsCV2/model_1.h5
563/563 [==============================] - 5408s 10s/step - loss: 0.6887 - accuracy: 0.5951 - val_loss: 0.6285 - val_accuracy: 0.6567
Epoch 2/30
563/563 [==============================] - ETA: 0s - loss: 0.6038 - accuracy: 0.6681
Epoch 2: val_accuracy improved from 0.65666 to 0.69646, saving model to /content/drive/MyDrive/Interface_E2/save_modelsCV2/model_1.h5
563/563 [==============================] - 289s 513ms/step - loss: 0.6038 - accuracy: 0.6681 - val_loss: 0.5812 - val_accuracy: 0.6965
Epoch 3/30
563/563 [==============================] - ETA: 0s - loss: 0.5677 - accuracy: 0.7024
Epoch 3: val_accuracy improved from 0.69646 to 0.70380, saving model to /content/drive/MyDrive/Interface_E2/save_mode

In [ ]:
validation_accuracy

[0.8378919363021851,
 0.8492663502693176,
 0.8554913401603699,
 0.8568252325057983,
 0.84989994764328]

In [ ]:
validation_loss



[0.3703453540802002,
 0.3328208327293396,
 0.33078640699386597,
 0.3352212905883789,
 0.34056127071380615]